In [1]:
import enum
import importlib
import inspect
import os
import shutil

from typing import List
from IPython.display import HTML
from sflkit.analysis.spectra import Line

from sflkit.analysis import analysis_type, factory
from sflkit import *
from sflkit.color import ColorCode
from sflkit import instrument_config, analyze_config
from sflkit.config import Config

from avicenna import *

## A faulty Program

First, we need a faulty program. We chose an implementation of the `middle(x, y, z)` function that returns the *middle* number of its three arguments. For example, `middle(1, 3, 2)` should return 2 because `1 < 2` and `2 < 3`. We introduced a fault in this implementation of `middle` that occurs in line 7 `m = y`. 

In [2]:
def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = y 
    else:
        if x > y:
            m = y
        elif x > z:
            m = x # desired line
    return m

In [3]:
import string
import csv
import logging

from pathlib import Path
#from avicenna import Avicenna
from debugging_framework.input.oracle import OracleResult
from isla.solver import ISLaSolver


We can now go through our event-files folder and iterate through each file, representing a test case. The relevant test cases will be denoted by an OracleResult.BUG for now. We can use this behavior-triggering input as our input for a run of Avicenna to find the constraints leading to this behavior.

The oracle used in this run of Avicenna will be the oracle checking against line-event calls. This requires us to run sflkit using Avicenna's new hypotheses after every run. 

- TODO : update Avicenna and fix up prototype for it 
- TODO : implement prototype functionality using SFLKit and Avicenna 0.2 

#### Testing inputs for regular middle bug. 

In [4]:
middle_grammar = {
    "<start>": ["<stmt>"],
    "<stmt>": ["<x>,<y>,<z>"],
    "<x>": ["<integer>"],
    "<y>": ["<integer>"],
    "<z>": ["<integer>"],
    "<integer>": ["<digit>", "<digit><integer>"],
    "<digit>": [str(num) for num in range(1, 10)]
}


middle_grammar_converted = {
    "<start>": ["<stmt>"],
    "<stmt>": ["str.to.int(<x>),str.to.int(<y>),str.to.int(<z>)"],
    "<x>": ["<integer>"],
    "<y>": ["<integer>"],
    "<z>": ["<integer>"],
    "<integer>": ["<digit>", "<digit><integer>"],
    "<digit>": [str(num) for num in range(1, 10)]
}

In [5]:
middle_inputs = ['2,3,1', '3,1,2', '2,1,3']

In [7]:
middle_py = 'middle.py'
tmp_py = 'tmp.py'
language='python'
predicates='line'
metrics='Tarantula'
#passing='event-files/0,event-files/1'
failing='event-files/2'
def get_config():
    return Config.create(path=middle_py,
                         working=tmp_py,
                         language=language,
                         predicates=predicates,
                         metrics=metrics,
                         failing=failing)
def instrument(out=True):
    instrument_config(get_config())
    if out:
        with open(tmp_py, 'r') as fp:
            print(fp.read())

In [8]:
instrument()

sflkit :: INFO     :: I found 11 events in middle.py.
sflkit :: INFO     :: I found 11 events in middle.py.


import sflkitlib.lib

def middle(x, y, z):
    sflkitlib.lib.add_line_event(0)
    m = z
    sflkitlib.lib.add_line_event(1)
    if y < z:
        sflkitlib.lib.add_line_event(2)
        if x < y:
            sflkitlib.lib.add_line_event(3)
            m = y
        else:
            sflkitlib.lib.add_line_event(4)
            if x < z:
                sflkitlib.lib.add_line_event(5)
                m = y
    else:
        sflkitlib.lib.add_line_event(6)
        if x > y:
            sflkitlib.lib.add_line_event(7)
            m = y
        else:
            sflkitlib.lib.add_line_event(8)
            if x > z:
                sflkitlib.lib.add_line_event(9)
                m = x
    sflkitlib.lib.add_line_event(10)
    return m


In [9]:
def analyzer_conf(conf: Config, factory):
    analyzer = Analyzer(irrelevant_event_files=conf.failing, 
                        relevant_event_files=conf.passing,
                        factory=factory)
    return analyzer

In [10]:
factory = factory.LineFactory()
line_event = analysis_type.AnalysisType(0)
line_event

LINE

In [11]:
analyzer = analyzer_conf(get_config(), factory=factory)

In [12]:
analyzer.analyze()

In [13]:
coverage: List[Line] = analyzer.get_coverage()
coverage = {line.line for line in coverage}
print(coverage)

{2, 3, 9, 10, 13}


In [14]:
def middle_inp_conv(inp):
    inp = inp.__str__()
    middle_input = inp.split(',')
    
    converted_inp = [
        int(middle_input[0]),
        int(middle_input[1]),
        int(middle_input[2])
    ]
    
    return converted_inp

In [15]:
from avicenna.avix import *
from avicenna.oracle_construction import * 
avix_oracle = construct_oracle(
                            program_under_test='middle',
                            inp_converter=middle_inp_conv,
                            timeout=10,
                            line = 7,
                            resource_path='rsc/',
                            program_oracle= None,
                            )


In [16]:
avix_oracle('2,1,3')

<OracleResult.FAILING: 'FAILING'>

In [17]:
def middle_oracle(x, y, z):
    # convert inputs
    
    sorted_inputs = [x, y, z]
    sorted_inputs.sort()
    return sorted_inputs[1]

In [18]:
regular_oracle = construct_oracle(
                            program_under_test=middle,
                            timeout=10,
                            program_oracle=middle_oracle,
                            default_oracle_result=OracleResult.PASSING)



In [19]:
param = list(map(int, str('2,1,3').strip().split(',')))
middle_inputs

['2,3,1', '3,1,2', '2,1,3']

In [20]:
regular_oracle('3,1,2')


<OracleResult.PASSING: 'PASSING'>

In [21]:
avicenna = Avicenna(
    grammar=middle_grammar,
    initial_inputs=middle_inputs,
    oracle=avix_oracle,
    max_iterations=10,
    top_n_relevant_features=3,
    max_conjunction_size=6,
)

In [22]:
avicenna2 = Avicenna(
    grammar=middle_grammar,
    initial_inputs=middle_inputs,
    oracle=regular_oracle,
    max_iterations=10,
    top_n_relevant_features=3, 
)

In [23]:
import warnings
# Suppress the specific SHAP warning
warnings.filterwarnings(
    "ignore",
    message="LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray",
)
warnings.filterwarnings(
    "ignore", 
    message="No further splits with positive gain, best gain: -inf"
)

In [24]:
logging.basicConfig(filename='avix.log', filemode='w', encoding='utf-8', level=logging.INFO, force=True)
# only 2 constraints used in the end why
best_invariant = avicenna.explain() # unparse with islaunparse for further use


KeyboardInterrupt: 

In [ ]:
logging.basicConfig(filename='avicenna.log', filemode='w', encoding='utf-8', level=logging.INFO, force=True)
#another_invariant = avicenna2.explain() # unparse with islaunparse for further use

In [ ]:
#print(another_invariant)

In [ ]:
print(best_invariant)
print(best_invariant[1])

(ConjunctiveFormula(ForallFormula(BoundVariable("elem_1", "<z>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_2", "<x>"), Constant("start", "<start>"), SMTFormula('(> (str.to_int elem_1) (str.to_int elem_2))', BoundVariable("elem_1", "<z>"), BoundVariable("elem_2", "<x>"), ))), ForallFormula(BoundVariable("elem_0", "<x>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_3", "<y>"), Constant("start", "<start>"), SMTFormula('(>= (str.to_int elem_0) (str.to_int elem_3))', BoundVariable("elem_0", "<x>"), BoundVariable("elem_3", "<y>"), )))), 1.0, 1.0)
1.0


In [ ]:
# solver1 = ISLaSolver(
#     grammar=middle_grammar,
#     formula=another_invariant[0],
#     enable_optimized_z3_queries=False,
# )

solver2 = ISLaSolver(
    grammar=middle_grammar,
    formula=best_invariant[0],
    enable_optimized_z3_queries=False,
)

results1 = []
results2 = []

# # print(best_invariant[0])
# # print(another_invariant[0])
# for _ in range(1,10):
#     results1.append(solver1.solve())
    
print("stop")
#    avix_oracle()
for _ in range(1,10):
    results2.append(solver2.solve())

stop


In [ ]:
for input in results1:
    if avix_oracle(input) == OracleResult.FAILING:
        print(input)
        print(avix_oracle(input))

In [ ]:
for input in results2:
    if avix_oracle(input) == OracleResult.FAILING:
        print(input)
        print(avix_oracle(input))
for input in results2:
    print(input)

2,2,4
FAILING
2,1,4
FAILING
4,4,8
FAILING
4,2,8
FAILING
4,3,8
FAILING
4,1,8
FAILING
1,1,8
FAILING
8,4,9
FAILING
8,2,9
FAILING
2,2,4
2,1,4
4,4,8
4,2,8
4,3,8
4,1,8
1,1,8
8,4,9
8,2,9


In [ ]:
solver = ISLaSolver(
    grammar=middle_grammar,
    formula=best_invariant[0],
    enable_optimized_z3_queries=False,
)

# this isnt working rn why not raaaaaaaaaaaaaaaaaaaaaaa
# should be inputs of type 2, 3, 1
for _ in range(1,10):
    print(solver.solve())


1,1,8
8,2,9
8,1,9
8,8,9
8,6,9
8,4,9
8,5,9
8,3,9
8,7,9


In [ ]:
# call func for middle, converts string input to usable integer values

def call_func_middle(inp: str):
    
    inp = inp.__str__()
    
    middle_input = inp.split(',')
    converted_inp =  [int(middle_input[0]), int(middle_input[1]), int(middle_input[2])]
    return converted_inp

In [ ]:
test_path = Path('rsc/')
print(Path.exists(test_path))


True


In [ ]:
from avicenna.avix import *
from avicenna.oracle_construction import * 

In [ ]:
def middle_inp_conv(inp):
    inp = inp.__str__()
    middle_input = inp.split(',')
    
    converted_inp = [
        int(middle_input[0]),
        int(middle_input[1]),
        int(middle_input[2])
    ]
    
    return converted_inp

In [ ]:
import tmp
importlib.reload(tmp)
tmp.sflkitlib.lib.reset()
avix_oracle = construct_oracle(
                            program_under_test='middle',
                            inp_converter=middle_inp_conv,
                            timeout=10,
                            line = 7,
                            resource_path='rsc/',
                            program_oracle= None)


In [ ]:
avix_oracle("6,3,7")

<OracleResult.FAILING: 'FAILING'>

In [ ]:
# # double check if this works

# AviX.create_event_file(instrumented_function='middle',
#                        #instr_path='tmp',
#                        inp = '2,1,3',
#                        conversion_func=middle_inp_conv,
#                        event_path='rsc/event_file'
#                        )

In [ ]:
avix = AviX(grammar=middle_grammar,
            initial_inputs=middle_inputs,
            oracle=avix_oracle,
            max_iterations=10,
            desired_line=7,
            put_path='middle.py',
            #instr_path='instrumented.py',
            min_precision = 0.7,)

In [ ]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer, DerivationTree

middle_grammar_converted = {
    "<start>": ["<stmt>"],
    "<stmt>": ["str.to.int(<x>),str.to.int(<y>),str.to.int(<z>)"],
    "<x>": ["<integer>"],
    "<y>": ["<integer>"],
    "<z>": ["<integer>"],
    "<integer>": ["<digit>", "<digit><integer>"],
    "<digit>": [str(num) for num in range(1, 10)]
}

fuzzer = GrammarFuzzer(middle_grammar)

for i in range(10):
    print(fuzzer.fuzz())

34146,7,89
3925,5,15
5985,6,1
3,4,5179
6,52,714477
2,6,8
78,2,486
114,496,2
71,8,298
76,7,2


https://github.com/uds-se/sflkit

<img src="qrcode.png" style="width:500px">